In [1]:
# Load and import general libraries

import pandas as pd
from sklearn import linear_model
from sklearn import tree
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
import seaborn as sns
import time
sns.set(style="white", color_codes=True)
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import LinearSVR

In [2]:
# read in file
data = pd.read_csv('diamonds.csv')

# drop the index column
data = data.drop(data.columns[0], axis=1)

In [3]:
# transform cut to int
def trans_cut(x):
    if x == "Fair": return 4
    if x == "Good": return 3
    if x == "Very Good": return 2
    if x == "Ideal": return 1
    if x == "Premium": return 0

In [4]:
# transform color into int
def trans_color(x):
    if x == "D": return 0
    if x == "E": return 1
    if x == "F": return 2
    if x == "G": return 3
    if x == "H": return 4
    if x == "I": return 5
    if x == "J": return 6

In [5]:
# transform clarity to int
def trans_clarity(x):
    if x == "I1": return 7
    if x == "SI1": return 6
    if x == "SI2": return 5
    if x == "VS1": return 4
    if x == "VS2": return 3
    if x == "VVS1": return 2
    if x == "VVS2": return 1
    if x == "IF": return 0

In [6]:
# replace current columns with new int versions
data['cut'] = data['cut'].apply(trans_cut)
data['color'] = data['color'].apply(trans_color)
data['clarity'] = data['clarity'].apply(trans_clarity)

In [7]:
# drop missing values
data = data[(data[['x', 'y', 'z']] != 0).all(axis=1)]

In [8]:
label_cut = LabelEncoder()
label_color = LabelEncoder()
label_clarity = LabelEncoder()
data['cut'] = label_cut.fit_transform(data['cut'])
data['color'] = label_color.fit_transform(data['color'])
data['clarity'] = label_clarity.fit_transform(data['clarity'])

In [9]:
# splitting to train and test
X = data.drop(['price'], axis=1)
y = data['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=309)

In [10]:
RSquared_Scores = []

In [11]:
def eval_model(model_type):
    start_time = time.time()
    model = model_type
    model.fit(X_train , y_train)
    cross_val_results = cross_val_score(estimator = model, X = X_train, y = y_train, cv = 5,verbose = 1)

    y_predict = model.predict(X_test)
    print('----------------------------------------------------------------------------')
    print(model_type)
    print('----------------------------------------------------------------------------')
    print('Results: %.2f' % model.score(X_test, y_test))
    print(cross_val_results)

    mse = mean_squared_error(y_test, y_predict)
    rmse = mean_squared_error(y_test, y_predict)**0.5
    r2 = r2_score(y_test, y_predict)
    mae = mean_absolute_error(y_test, y_predict)

    print()
    print('MSE    : %0.2f ' % mse)
    print('RMSE   : %0.2f ' % rmse)
    print('R2     : %0.2f ' % r2)
    print('MAE    : %0.2f ' % mae)
    print("--- %s seconds ---" % (time.time() - start_time))
    print('\n\n')

    RSquared_Scores.append(r2)

In [12]:
def ScaledSGDRegression():
    start_time = time.time()

    scaler = StandardScaler()
    scaler.fit(X)
    Xs = scaler.transform(X)
    X_train, X_test, y_train, y_test = train_test_split(Xs, y, test_size=0.3, random_state=309)
    sgd = linear_model.SGDRegressor()
    sgd.fit(X_train, y_train)


    y_pred=sgd.predict(X_test)
    print('Results: %.2f' % sgd.score(X_test, y_test))

    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred)**0.5
    r2 = r2_score(y_test, y_pred)

    print()
    print('MSE    : %0.2f ' % mse)
    print('RMSE   : %0.2f ' % rmse)
    print('R2     : %0.2f ' % r2)
    print('MAE    : %0.2f ' % mae)
    print("--- %s seconds ---" % (time.time() - start_time))
    print('\n\n')

In [13]:
def ScaledSVR():
    start_time = time.time()


    print('###### Support Vector Regression (SVR) #######')
    scaler = StandardScaler()
    scaler.fit(X)
    Xs = scaler.transform(X)
    X_train, X_test, y_train, y_test = train_test_split(Xs, y, test_size=0.3, random_state=309)


    #Then we fit the regressor to the scaled dataset :

    #fitting the SVR to the dataset

    regressor = SVR(kernel='rbf',C=100)
    regressor.fit(X_train, y_train)
    y_pred=regressor.predict(X_test)

    print('')
    print('Score : %.4f' % regressor.score(X_test, y_test))

    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred)**0.5
    r2 = r2_score(y_test, y_pred)

    print('')
    print('MSE    : %0.2f ' % mse)
    print('MAE    : %0.2f ' % mae)
    print('RMSE   : %0.2f ' % rmse)
    print('R2     : %0.2f ' % r2)
    print("--- %s seconds ---" % (time.time() - start_time))
    print('\n\n')


In [14]:
def main():
    eval_model(LinearRegression())
    eval_model(KNeighborsRegressor())
    eval_model(Ridge(alpha=1.0))
    eval_model(tree.DecisionTreeClassifier())
    ScaledSGDRegression()
    ScaledSVR()
    eval_model(RandomForestRegressor())
    eval_model(GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=1, random_state=309, loss='ls', verbose=1))
    eval_model(LinearSVR(random_state=309, tol=1e-5))
    eval_model(MLPRegressor(max_iter=1000))

In [ ]:
main()

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.1s finished


----------------------------------------------------------------------------
LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)
----------------------------------------------------------------------------
Results: 0.90
[0.89960819 0.88622343 0.89116416 0.88766085 0.88709594]

MSE    : 1631602.42 
RMSE   : 1277.34 
R2     : 0.90 
MAE    : 839.32 
--- 0.21569132804870605 seconds ---





[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    6.4s finished


----------------------------------------------------------------------------
KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                    weights='uniform')
----------------------------------------------------------------------------
Results: 0.95
[0.94658    0.9402005  0.94412738 0.94430429 0.94733194]

MSE    : 834834.37 
RMSE   : 913.69 
R2     : 0.95 
MAE    : 493.63 
--- 11.252387762069702 seconds ---



----------------------------------------------------------------------------
Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)
----------------------------------------------------------------------------
Results: 0.90
[0.89957876 0.88678949 0.89118191 0.88769838 0.88710878]

MSE    : 1632206.75 
RMSE   : 1277.58 
R2     : 0.90 
MAE    : 839.73 
--- 0.13843178749084473 seconds ---





[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/home/karu/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  4.6min finished


----------------------------------------------------------------------------
DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')
----------------------------------------------------------------------------
Results: 0.15
[0.0927881  0.1314208  0.17173276 0.2021298  0.23699278]

MSE    : 1265370.00 
RMSE   : 1124.89 
R2     : 0.92 
MAE    : 512.00 
--- 344.23788952827454 seconds ---



Results: 0.90

MSE    : 1660487.66 
RMSE   : 1288.60 
R2     : 0.90 
MAE    : 839.26 
--- 0.6043992042541504 seconds ---



###### Support Vector Regression (SVR) #######


/home/karu/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
